In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))
sys.path.append(project_root)

print(project_root)

import machine_learning.files_lib as FL
import machine_learning.ml_lib as ML
from data_extractor.data_extractor import load_df, extract_X_y_from_df 

In [ ]:
df = load_df("../../data/")
# Filter the data and add the column 'label' depends what you need
df = df.query("desired_answer == answer and data_type in ['REAL', 'FAKE']")
df['label'] = df.apply(lambda x: 1 if x.block_no in [1,3] else 0, axis = 1)
df.head()

In [ ]:
X,y = extract_X_y_from_df(df)

In [2]:
param_grid = {
    'classifier__C': [0.1, 1, 10],
    'classifier__kernel': ['linear', 'rbf'],
    'classifier__gamma': ['scale', 'auto'],
    'classifier__class_weight': [None, 'balanced']
}

result_folder = "../results/svc_base/"

In [ ]:
from sklearn.svm import SVC

best_model, best_params, test_score = ML.feature_extraction_grid_search(
    X, y, param_grid, test_size=0.2, cv=5, scoring='accuracy', result_folder=result_folder, model_name='svc', classifier=SVC
)

In [ ]:
from datetime import datetime

f"{datetime.now().strftime('%Y-%m-%d_%H:%M:%S_%f')}"

In [8]:
import machine_learning.visualisation_lib as VL

In [ ]:
file_path = "/Users/gsk/documents/neuron/lie-detector-ai/gs/results/svc/_2024-10-05_22:50:04/result_svc_grid_search_random_forest_eeg_5fold_1728162160.json"
cv_results_df, best_params, best_score, test_score, classification_report = FL.read_grid_search_results(file_path)

print("Best Parameters:")
print(best_params)
print(f"\nBest Cross-Validation Score: {best_score:.4f}")
print(f"Test Score: {test_score:.4f}\n")

VL.plot_mean_test_scores(cv_results_df, 'classifier__max_depth')
VL.plot_mean_test_scores(cv_results_df, 'classifier__n_estimators')
VL.plot_mean_test_scores(cv_results_df, 'classifier__min_samples_split')

VL.plot_heatmap_mean_test_scores(cv_results_df, 'classifier__n_estimators', 'classifier__max_depth')

VL.plot_fit_and_score_times(cv_results_df, 'classifier__n_estimators')

if classification_report:
    print("Classification Report:")
    for label, metrics in classification_report.items():
        if isinstance(metrics, dict):
            if label in ['accuracy', 'macro avg', 'weighted avg']:
                print(f"\n{label}:")
                for metric_name, metric_value in metrics.items():
                    print(f"  {metric_name}: {metric_value:.4f}")
            else:
                print(f"\nClass {label}:")
                for metric_name, metric_value in metrics.items():
                    print(f"  {metric_name}: {metric_value:.4f}")


In [ ]:
X.shape, y.shape

In [38]:
param_grid = [
        (SVC(),  {
    'model__C': [0.1, 1, 10],
    'model__kernel': ['linear', 'rbf'],
    'model__gamma': ['scale', 'auto'],
    'model__class_weight': [None, 'balanced']
}),
]

In [ ]:
from sklearn.svm import SVC

best_model, best_params, test_score = ML.grid_search_multiple_models(
     param_grid,X, y, test_size=0.2, cv=5, scoring='accuracy', result_folder=result_folder
)